Once training is complete and the model weights have been saved, we can generate text.

In [1]:
import numpy as np                                                                                           
import sys                                                                                                   
import os                                                                                                    
import json                                                                                                  
from tqdm import tqdm                                                                                        
from keras.models import load_model                                                                          
from keras.models import model_from_json                                                                     
from keras import backend as K                                                                               
import pickle     

/home/csci4850/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def dist(x,y):
    s = 0
    z = x-y

    for element in z:
        s += element**2
    return np.sqrt(s)

def closest(dictionary, vec):
    min_dist = 1000000000000
    for key,val in dictionary.items():
        v = np.array(val)[0]

        d = dist(v, vec)

        if d < min_dist:
            min_dist = d
            closest = key
            closest_vec = val
    return closest, np.array(closest_vec)

In [3]:
########## SET DIRECTORIES ##########
DATA_DIR = os.path.join("data", "train", "cleaned")
MAPPING_FILE = os.path.join("utils", "mapping.pkl")
RNN_MODEL = os.path.join("models", "rnn_model.hdf5")

INCLUDE_POS = True 
NUM_POS_TAGS = 47

########## IMPORT DATA ##########
with open(MAPPING_FILE, 'rb') as f:
    mapping = pickle.load(f)

In [4]:
########## LOAD MODEL ##########
model = load_model(RNN_MODEL)

In [9]:
########## INIT ##########

# set up start token
token = mapping['ST']
token = np.array(token)
token = np.reshape(token, (1,) + token.shape)

if INCLUDE_POS:
    final_shape = token.shape[-1] + NUM_POS_TAGS 
else:
    final_shape = token.shape[-1]

tmp = np.zeros(shape=(1,1,final_shape))
tmp[0,0,:len(token[0,0])] = token[0,0,:]
token = tmp
noise = np.random.rand(token.shape[0], token.shape[1], token.shape[2])
noise /= 10 #small amount of noise

print(token.shape)
print(noise.shape)

en_count = 0

words = []
words.append('ST')

(1, 1, 260)
(1, 1, 260)


In [10]:
########## GENERATE WORDS ##########

print('ST', end=' ')

while en_count <= 50:
    out = model.predict([token, noise])

    # snap the network's prediction to the closest real word, and also
    # snap the network's prediction to the closest vector in our space
    # so that it predicts with real words as previous values
    closest_word, closest_vec = closest(mapping, out[0,0,:])
    token = np.zeros(shape=out.shape)
    token[0,0,:] = closest_vec

    # fix shapes
    tmp = np.zeros(shape=(1,1,final_shape))
    tmp[0,0,:len(out[0,0])] = out[0,0,:]
    out = tmp

    tmp = np.zeros(shape=(1,1,final_shape))
    tmp[0,0,:len(token[0,0])] = token[0,0,:]
    token = tmp

    noise = np.random.rand(token.shape[0], token.shape[1], token.shape[2])
    noise /= 10

    words.append(closest_word)
    
    if closest_word == "EN":
        en_count += 1
        print(closest_word)
    else:
        print(closest_word, end=' ')

ST EN
pickle. a into myself turned i ST EN
counseling? a into myself turned i ST EN
pickle. a into i'm ST EN
pickle. a into myself i ST EN
counseling? of ST EN
pickle. a into myself turned i ST EN
pickle. a into myself myself turned i ST EN
pickle. a into myself turned you do turned you do turned i if a into myself turned you do would do would ST EN
this. do would ST EN
pickle. a into myself i'm ST EN
pickle. a into myself turned ST EN
pickle. a into myself i ST EN
pickle. a into myself turned i ST EN
pickle! a i'm ST EN
pickle. a into myself turned you i ST EN
pickle. a into would ST EN
pickle. a into myself myself myself myself turned i ST EN
pickle. a into myself turned you do would ST EN
man. a into would do turned i ST EN
pickle. a into would ST EN
pickle. a into myself turned i ST EN
man. a into myself turned you ST EN
pickle. i'm ST EN
morty! pickle. a into myself would do turned i ST EN
pickle. a into myself turned i ST EN
pickle! and a into myself turned i if ST EN
pickle. a i

In [ ]:
INCLUDE_POS = False 

In [9]:
########## INIT ##########

# set up start token
token = mapping['ST']
token = np.array(token)
token = np.reshape(token, (1,) + token.shape)

if INCLUDE_POS:
    final_shape = token.shape[-1] + NUM_POS_TAGS 
else:
    final_shape = token.shape[-1]

tmp = np.zeros(shape=(1,1,final_shape))
tmp[0,0,:len(token[0,0])] = token[0,0,:]
token = tmp
noise = np.random.rand(token.shape[0], token.shape[1], token.shape[2])
noise /= 10 #small amount of noise

print(token.shape)
print(noise.shape)

en_count = 0

words = []
words.append('ST')

(1, 1, 260)
(1, 1, 260)


In [10]:
########## GENERATE WORDS ##########

print('ST', end=' ')

while en_count <= 50:
    out = model.predict([token, noise])

    # snap the network's prediction to the closest real word, and also
    # snap the network's prediction to the closest vector in our space
    # so that it predicts with real words as previous values
    closest_word, closest_vec = closest(mapping, out[0,0,:])
    token = np.zeros(shape=out.shape)
    token[0,0,:] = closest_vec

    # fix shapes
    tmp = np.zeros(shape=(1,1,final_shape))
    tmp[0,0,:len(out[0,0])] = out[0,0,:]
    out = tmp

    tmp = np.zeros(shape=(1,1,final_shape))
    tmp[0,0,:len(token[0,0])] = token[0,0,:]
    token = tmp

    noise = np.random.rand(token.shape[0], token.shape[1], token.shape[2])
    noise /= 10

    words.append(closest_word)
    
    if closest_word == "EN":
        en_count += 1
        print(closest_word)
    else:
        print(closest_word, end=' ')

ST EN
pickle. a into myself turned i ST EN
counseling? a into myself turned i ST EN
pickle. a into i'm ST EN
pickle. a into myself i ST EN
counseling? of ST EN
pickle. a into myself turned i ST EN
pickle. a into myself myself turned i ST EN
pickle. a into myself turned you do turned you do turned i if a into myself turned you do would do would ST EN
this. do would ST EN
pickle. a into myself i'm ST EN
pickle. a into myself turned ST EN
pickle. a into myself i ST EN
pickle. a into myself turned i ST EN
pickle! a i'm ST EN
pickle. a into myself turned you i ST EN
pickle. a into would ST EN
pickle. a into myself myself myself myself turned i ST EN
pickle. a into myself turned you do would ST EN
man. a into would do turned i ST EN
pickle. a into would ST EN
pickle. a into myself turned i ST EN
man. a into myself turned you ST EN
pickle. i'm ST EN
morty! pickle. a into myself would do turned i ST EN
pickle. a into myself turned i ST EN
pickle! and a into myself turned i if ST EN
pickle. a i